In [1]:
from utils import Load_Rumours_Dataset_filtering_since_first_post
import numpy as np
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
# Usage
file_path_replies = r"replies_charlie_hebdo.pkl"
file_path_posts = r"posts_charlie_hebdo.pkl"

In [3]:
processor = Load_Rumours_Dataset_filtering_since_first_post(file_path_replies, file_path_posts, time_cut=2630)
processor.load_data()
processor.process_data()
train,test= processor.get_final_dataframes()


In [4]:
X_train  = train.drop(columns=['rumour'])
X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])


X_test  = test.drop(columns=['rumour'])
X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])

#X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y_train =train['rumour']
y_test =test['rumour']

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset



# Model definition
class RumorDetectionLSTM(nn.Module):
    def __init__(self, embedding_dim=100, lstm_hidden_size=32, dense_hidden_size=16):
        super(RumorDetectionLSTM, self).__init__()
        
        # LSTM for the 100-dimensional embeddings
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_size, batch_first=True)
        
        # Dense layers for other features
        self.dense1 = nn.Linear(8, 16)  # 8 non-embedding features
        self.dense2 = nn.Linear(16, dense_hidden_size)
        
        # Combine LSTM and dense features
        self.fc1 = nn.Linear(lstm_hidden_size + dense_hidden_size, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        # Separate embeddings and other features
        embeddings = x[:, -100:].unsqueeze(1)  # (batch, seq_len=1, embedding_dim)
        other_features = x[:, :8]  # First 8 features
        
        # LSTM output
        lstm_out, _ = self.lstm(embeddings)
        lstm_out = lstm_out[:, -1, :]  # Get the last LSTM output
        
        # Dense layers for other features
        dense_out = torch.relu(self.dense1(other_features))
        dense_out = torch.relu(self.dense2(dense_out))
        
        # Concatenate LSTM and dense outputs
        combined = torch.cat((lstm_out, dense_out), dim=1)
        
        # Fully connected layers for classification
        x = torch.relu(self.fc1(combined))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()


In [8]:
# Assuming X_train, X_test, y_train, and y_test are available as numpy arrays
# Convert them to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Dataset and DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

/tmp/ipykernel_8172/3736111541.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
/tmp/ipykernel_8172/3736111541.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
/tmp/ipykernel_8172/3736111541.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
/tmp/ipykernel_8172/3736111541.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() 

In [9]:
from sklearn.metrics import recall_score, precision_score

# Model, criterion, optimizer initialization (as before)
model = RumorDetectionLSTM()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with loss and recall monitoring
epochs = 100  # Adjust as needed
train_recall_interval = 50  # Calculate train recall every 10 epochs
loss_interval = 50  # Print loss every 10 epochs

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Print loss every 10 epochs
    if (epoch + 1) % loss_interval == 0:
        model.eval()
        train_preds = []
        train_labels = []
        with torch.no_grad():
            for X_batch, y_batch in train_loader:
                output = model(X_batch)
                preds = (output >= 0.5).int()  # Binarize predictions
                train_preds.extend(preds.tolist())
                train_labels.extend(y_batch.tolist())
        
        train_recall = recall_score(train_labels, train_preds)
        train_precision = precision_score(train_labels, train_preds)
        
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss / len(train_loader):.4f},\
              Train Precision: {train_precision:.4f},Train Recall: {train_recall:.4f}")
    


# Final evaluation on test set with recall and precision
model.eval()
test_preds = []
test_labels = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        preds = (output >= 0.5).int()  # Binarize predictions
        test_preds.extend(preds.tolist())
        test_labels.extend(y_batch.tolist())

# Calculate final test recall and precision
test_recall = recall_score(test_labels, test_preds)
test_precision = precision_score(test_labels, test_preds)

print(f"Final Test Recall: {test_recall:.4f}")
print(f"Final Test Precision: {test_precision:.4f}")


Epoch 50, Train Loss: 0.1183,              Train Precision: 0.9393,Train Recall: 0.8403
Epoch 100, Train Loss: 0.0158,              Train Precision: 0.9905,Train Recall: 1.0000
Final Test Recall: 0.7333
Final Test Precision: 0.6875


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Model, criterion, optimizer initialization (as before)
model = RumorDetectionLSTM()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 200
loss_interval = 50

# Save train outputs for threshold tuning
train_probs_all = []
train_labels_all = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    # Store train outputs for final threshold selection
    if (epoch + 1) == epochs:
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in train_loader:
                output = model(X_batch)
                train_probs_all.extend(output.squeeze().tolist())
                train_labels_all.extend(y_batch.squeeze().tolist())
    
    # Logging
    if (epoch + 1) % loss_interval == 0:
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss / len(train_loader):.4f}")

# ------------------------
# Find best threshold on train set
# ------------------------
train_probs_all = np.array(train_probs_all)
train_labels_all = np.array(train_labels_all)

best_thresh = 0.5
best_f1 = 0.0
for t in np.arange(0.0, 1.01, 0.01):
    preds = (train_probs_all >= t).astype(int)
    f1 = f1_score(train_labels_all, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"\nBest Threshold on Train (max F1): {best_thresh:.2f} | F1: {best_f1:.4f}")

# ------------------------
# Final evaluation on test set using best threshold
# ------------------------
model.eval()
test_probs = []
test_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        test_probs.extend(output.squeeze().tolist())
        test_labels.extend(y_batch.squeeze().tolist())

test_probs = np.array(test_probs)
test_labels = np.array(test_labels)
test_preds = (test_probs >= best_thresh).astype(int)

# Calculate final metrics
test_recall = recall_score(test_labels, test_preds)
test_precision = precision_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds)

print(f"Final Test Recall: {test_recall:.4f}")
print(f"Final Test Precision: {test_precision:.4f}")
print(f"Final Test F1: {test_f1:.4f}")


Epoch 50, Train Loss: 0.1344
Epoch 100, Train Loss: 0.0401
Epoch 150, Train Loss: 0.0059
Epoch 200, Train Loss: 0.0040

Best Threshold on Train (max F1): 0.15 | F1: 1.0000
Final Test Recall: 0.8000
Final Test Precision: 0.6486
Final Test F1: 0.7164


In [22]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("LSTM 2025-07-16 Charlie Hebdo")

2025/07/15 20:13:37 INFO mlflow.tracking.fluent: Experiment with name 'LSTM 2025-07-16 Charlie Hebdo' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/rumour-detection-gnn/mlruns/99', creation_time=1752610417721, experiment_id='99', last_update_time=1752610417721, lifecycle_stage='active', name='LSTM 2025-07-16 Charlie Hebdo', tags={}>

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score
import mlflow

file_path_replies = r"replies_charlie_hebdo.pkl"
file_path_posts = r"posts_charlie_hebdo.pkl"

for time_cut in range(50, 24 * 3 * 60, 10):
    print(f"\nProcessing time_cut: {time_cut}")

    dataset_generator = Load_Rumours_Dataset_filtering_since_first_post(
        file_path_replies, file_path_posts, time_cut=time_cut)
    dataset_generator.load_data()
    dataset_generator.process_data()
    train_df, test_df = dataset_generator.get_final_dataframes()

    X_train = train_df.drop(columns=['rumour'])
    X_train = np.hstack([
        X_train.drop(columns=['embeddings_avg']).values,
        np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))
    ])
    y_train = train_df['rumour']

    X_test = test_df.drop(columns=['rumour'])
    X_test = np.hstack([
        X_test.drop(columns=['embeddings_avg']).values,
        np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))
    ])
    y_test = test_df['rumour']

    # Handle class imbalance
    num_pos = sum(y_train)
    num_neg = len(y_train) - num_pos
    pos_weight = torch.tensor([num_neg / num_pos], dtype=torch.float32)

    # Convert to tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test.values, dtype=torch.float32)

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    # Define model, loss, and optimizer
    model = RumorDetectionLSTM()  # <- define your model class elsewhere
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 200
    loss_interval = 50

    train_probs_all = []
    train_labels_all = []

    with mlflow.start_run():
        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                output = model(X_batch).view(-1)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            if (epoch + 1) % loss_interval == 0:
                print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss / len(train_loader):.4f}")

            # Store outputs for threshold tuning after last epoch
            if (epoch + 1) == epochs:
                model.eval()
                with torch.no_grad():
                    for X_batch, y_batch in train_loader:
                        output = model(X_batch).view(-1)
                        train_probs_all.extend(output.tolist())
                        train_labels_all.extend(y_batch.tolist())

        # Threshold tuning (maximize F1)
        train_probs_all = np.array(train_probs_all)
        train_labels_all = np.array(train_labels_all)
        best_thresh = 0.5
        best_f1 = 0.0
        for t in np.arange(0.0, 1.01, 0.01):
            preds = (train_probs_all >= t).astype(int)
            f1 = f1_score(train_labels_all, preds)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = t

        print(f"\nBest Threshold on Train (max F1): {best_thresh:.2f} | F1: {best_f1:.4f}")

        # Final test evaluation
        model.eval()
        test_probs = []
        test_labels = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                output = model(X_batch).view(-1)
                test_probs.extend(output.tolist())
                test_labels.extend(y_batch.tolist())

        test_probs = np.array(test_probs)
        test_labels = np.array(test_labels)
        test_preds = (test_probs >= best_thresh).astype(int)

        test_recall = recall_score(test_labels, test_preds)
        test_precision = precision_score(test_labels, test_preds)
        test_f1 = f1_score(test_labels, test_preds)
        test_auc = roc_auc_score(test_labels, test_probs)

        print(f"Final Test Recall: {test_recall:.4f}")
        print(f"Final Test Precision: {test_precision:.4f}")
        print(f"Final Test F1: {test_f1:.4f}")
        print(f"Final Test AUC: {test_auc:.4f}")

        # MLflow logging
        mlflow.log_metric("train_f1", best_f1)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_f1", test_f1)
        mlflow.log_metric("test_auc", test_auc)
        mlflow.log_metric("time_cut", time_cut)
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("threshold", best_thresh)



Processing time_cut: 50
Epoch 50, Train Loss: 0.1167
Epoch 100, Train Loss: 0.0251
Epoch 150, Train Loss: 0.0037
Epoch 200, Train Loss: 0.0008

Best Threshold on Train (max F1): 0.12 | F1: 1.0000
Final Test Recall: 0.8889
Final Test Precision: 0.8000
Final Test F1: 0.8421
Final Test AUC: 0.8907

Processing time_cut: 60
Epoch 50, Train Loss: 0.1295
Epoch 100, Train Loss: 0.0409
Epoch 150, Train Loss: 0.0054
Epoch 200, Train Loss: 0.0007

Best Threshold on Train (max F1): 0.06 | F1: 1.0000
Final Test Recall: 0.9524
Final Test Precision: 0.8696
Final Test F1: 0.9091
Final Test AUC: 0.8929

Processing time_cut: 70
Epoch 50, Train Loss: 0.1378
Epoch 100, Train Loss: 0.0420
Epoch 150, Train Loss: 0.0041
Epoch 200, Train Loss: 0.0015

Best Threshold on Train (max F1): 0.05 | F1: 1.0000
Final Test Recall: 0.9048
Final Test Precision: 0.7917
Final Test F1: 0.8444
Final Test AUC: 0.8753

Processing time_cut: 80
Epoch 50, Train Loss: 0.1267
Epoch 100, Train Loss: 0.0377
Epoch 150, Train Loss: 0.